In [1]:
import torch
from torchvision import models
import os

### Inspecting the checkpoint

In [2]:
checkpoint = torch.load('./checkpoint/coffee-diseases-res50-4-ckpt.t7', map_location='cpu')

keys = checkpoint.keys()
print(keys)

print(checkpoint['acc'])
print(checkpoint['epoch'])
# print(checkpoint['scaler'])

dict_keys(['model', 'optimizer', 'scaler', 'acc', 'epoch'])
72.38924050632912
10


### Loading the net

In [14]:
net = models.resnet50(weights='DEFAULT')
print(net)
num_features = net.fc.in_features
net.fc = torch.nn.Sequential(
    torch.nn.Linear(num_features, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 4),
    torch.nn.Softmax(dim=1)
)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
# Load the checkpoint
checkpoint = torch.load('./checkpoint/coffee-diseases-res50-4-ckpt.t7', map_location='cpu')

print(checkpoint['model'].items())

odict_items([('module.conv1.weight', tensor([[[[-6.1085e-03, -2.5423e-03,  3.8347e-02,  ...,  4.7431e-02,
           -2.0598e-02,  8.4422e-03],
          [-5.5432e-02,  4.6195e-02,  7.9482e-02,  ...,  8.9382e-02,
            3.1383e-02, -5.6589e-02],
          [ 7.0539e-02, -2.6849e-01,  4.0613e-01,  ..., -1.6245e-01,
            2.2153e-01, -7.0700e-02],
          ...,
          [-1.0788e-01,  3.8159e-01, -4.5420e-01,  ...,  6.8450e-01,
           -5.7727e-01,  2.2514e-01],
          [ 2.8164e-02, -1.7506e-01,  6.4627e-01,  ...,  5.2979e-01,
           -4.5688e-02, -6.5161e-02],
          [ 4.7953e-02, -1.2890e-01,  2.0043e-02,  ..., -3.5478e-01,
            1.9298e-01, -1.9800e-02]],

         [[ 1.0157e-02,  4.5673e-03, -1.5380e-02,  ...,  8.5289e-02,
           -6.6679e-02,  1.5497e-02],
          [-3.1610e-02,  3.9957e-02,  8.7883e-02,  ...,  1.1710e-01,
            2.2308e-02, -4.3662e-02],
          [ 5.7362e-02, -2.6282e-01,  4.2881e-01,  ..., -2.3584e-01,
            3.2171e-0

In [56]:
# Load the checkpoint
checkpoint = torch.load('./checkpoint/coffee-diseases-res50-4-ckpt.t7', map_location='cpu')

keys = checkpoint.keys()
print(keys)

# Remove the "module" prefix from the keys
new_state_dict = {k.replace('module.', ''): v for k, v in checkpoint['model'].items()}

# Load the modified state_dict into the model
net.load_state_dict(new_state_dict)

dict_keys(['model', 'optimizer', 'scaler', 'acc', 'epoch'])


<All keys matched successfully>

In [36]:
# print('==> loading from checkpoint..')
# assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
# checkpoint = torch.load('./checkpoint/res50-4-ckpt.t7')

# keys = checkpoint.keys()
# print(keys)

# net.load_state_dict(checkpoint['model'])
# best_acc = checkpoint['acc']
# start_epoch = checkpoint['epoch']

# net = ResNet50()
# checkpoint = torch.load('./checkpoint/coffee-diseases-res50-4-ckpt.t7')
# keys = checkpoint.keys()
# print(keys)
# net.load_state_dict(checkpoint['model'])


# net = checkpoint['net']
# print(net)
# net.load_state_dict(checkpoint['net'])

### Saving the .pth model

In [57]:
torch.save(net.state_dict(), './checkpoint/resnet50.pth')

### Loading the model

In [59]:
pthPath = f'./checkpoint/resnet50.pth'
onnxPath = f'./checkpoint/resnet50.onnx'

# model =  torch.load(pthPath)
# model_dict = model.state_dict()
# net.load_state_dict(model_dict)

### Saving the onnx

In [60]:
!pip install onnx

In [61]:
import onnx
from torch.autograd import Variable

# dummy_input = torch.randn(1, 3, 32, 32)
dummy_input = torch.randn(1, 3, 60, 60)

# Export the model
torch.onnx.export(net,                 # model being run
                  dummy_input,                         # model input (or a tuple for multiple inputs)
                  onnxPath,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

